In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr, ttest_ind
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import logging
import os

DATA_DIR = "data/"
DATA_PROC_DIR = "data-processed/"
ATAC_SEQ_PATH = os.path.join(DATA_DIR, "ImmGenATAC18_AllOCRsInfo.csv")
RNA_SEQ_PATH = os.path.join(DATA_DIR, "mmc2.csv")
PROC_PEAKS_PATH = os.path.join(DATA_PROC_DIR, "peaks_annotated.csv")

# ——————————————————————————————————————————————————————————————
# 1) Load & align ATAC & RNA
# ——————————————————————————————————————————————————————————————

atac = pd.read_csv(ATAC_SEQ_PATH, index_col=[0,1,2]).drop(columns=["mm10.60way.phastCons_scores", "_-log10_bestPvalue", "Included.in.systematic.analysis"])
# keep only numeric columns (cell types)
atac = atac.select_dtypes(include=[np.number])

rna = pd.read_csv(RNA_SEQ_PATH, index_col=0)

# intersect & reorder
common = atac.columns.intersection(rna.columns)
atac = atac[common]
rna  = rna[common]

# ——————————————————————————————————————————————————————————————
# 2) Load peak annotation & build gene→peak map
# ——————————————————————————————————————————————————————————————

peaks = pd.read_csv(PROC_PEAKS_PATH)
peaks["gene_list"] = peaks["genes.within.100Kb"].str.split(",") # multiple genes per peak
pe = peaks.explode("gene_list").rename(columns={"gene_list":"gene"})
pe = pe[pe["gene"].notna()]
gene2peaks = pe.groupby("gene")["id"].apply(list).to_dict()
#Alternatively, if we would want to use the TSS_GeneName field: (meaning only one peak per gene)
#gene2peaks = peaks.groupby("TSS_GeneName")["id"].apply(list).to_dict()

# ——————————————————————————————————————————————————————————————
# 3) Define our lineage
# ——————————————————————————————————————————————————————————————

lineage_cells = ["NK.27+11b-.BM","NK.27+11b+.BM","NK.27-11b+.BM","NK.27+11b-.Sp","NK.27+11b+.Sp","NK.27-11b+.Sp","ILC2.SI", "ILC3.NKp46-CCR6-.SI", "ILC3.NKp46+.SI", "ILC3.CCR6+.SI"]

# ——————————————————————————————————————————————————————————————
# 4) Prepare for modeling
# ——————————————————————————————————————————————————————————————

model = LinearRegression()
records = []      # will hold regression + correlation results

# ——————————————————————————————————————————————————————————————
# 5) Loop over genes
# ——————————————————————————————————————————————————————————————

for gene, peak_ids in gene2peaks.items():
    # require gene in rna & ≥2 CREs
    if gene not in rna.index or len(peak_ids) < 2:
        continue

    try:
        sub = atac.loc[peak_ids]  # rows=CREs × cols=cell types
    except KeyError:
        continue
    # drop CREs with any missing data
    sub = sub.dropna(axis=0, how="any")
    if sub.shape[0] < 2:
        continue

    # global regression
    Xg = sub.values.T   # shape: samples × features
    yg = rna.loc[gene].values
    model.fit(Xg, yg)
    r2g = model.score(Xg, yg)
    betag = model.coef_

    # lineage regression
    sub_l = sub[lineage_cells].dropna(axis=0, how="any")
    if sub_l.shape[0] < 2:
        continue
    Xl = sub_l.values.T
    yl = rna.loc[gene, lineage_cells].values
    model.fit(Xl, yl)
    r2l = model.score(Xl, yl)
    betal = model.coef_

    # record per CRE
    for idx, pid in enumerate(sub.index):
        # correlation
        x = atac.loc[pid]
        r, p = pearsonr(x, rna.loc[gene])
        # effect sign
        eff = "activating" if betag[idx] > 0 else "repressing"
        records.append({
            "gene":          gene,
            "id":            pid[0],
            "R2_global":     r2g,
            "R2_lineage":    r2l,
            "beta_global":   betag[idx],
            "beta_lineage":  betal[idx] if idx < len(betal) else np.nan,
            "pearson_r":     r,
            "effect":        eff
        })

# ——————————————————————————————————————————————————————————————
# 6) Compile into DataFrame
# ——————————————————————————————————————————————————————————————

df = pd.DataFrame(records)
df["beta_diff"] = df["beta_lineage"] - df["beta_global"]

# merge location annotations
loc = peaks[["id","is_promoter","is_intragenic"]]
df = df.merge(loc, on="id")

# save
df.to_csv("data-processed/cre_gene_regression_complete.csv", index=False)

                  gene                                       peak_id  \
0        0610005C13Rik  (ImmGenATAC1219.peak_427187, chr7, 45477207)   
1        0610005C13Rik  (ImmGenATAC1219.peak_427188, chr7, 45477517)   
2        0610005C13Rik  (ImmGenATAC1219.peak_427189, chr7, 45478263)   
3        0610005C13Rik  (ImmGenATAC1219.peak_427190, chr7, 45481604)   
4        0610005C13Rik  (ImmGenATAC1219.peak_427191, chr7, 45488432)   
...                ...                                           ...   
1234986          l7Rn6  (ImmGenATAC1219.peak_435106, chr7, 90039119)   
1234987          l7Rn6  (ImmGenATAC1219.peak_435107, chr7, 90039677)   
1234988          l7Rn6  (ImmGenATAC1219.peak_435108, chr7, 90039953)   
1234989          l7Rn6  (ImmGenATAC1219.peak_435109, chr7, 90040662)   
1234990          l7Rn6  (ImmGenATAC1219.peak_435110, chr7, 90041097)   

         R2_global  R2_lineage  beta_global  beta_lineage  pearson_r  \
0         1.000000         1.0     0.032675      0.001360   0.0

KeyError: 'id'

In [ ]:
# ——————————————————————————————————————————————————————————————
# 7) Answering Your Questions
# ——————————————————————————————————————————————————————————————

# a) Variance explained per gene (global)
r2_global = df.groupby("gene")["R2_global"].first()
print("R²_global summary:\n", r2_global.describe())

# b) Coefficient differences (lineage vs global)
print("β difference summary:\n", df["beta_diff"].describe())

# c) Which CREs control lineage-specific genes?
#    Define by top delta R2 = R2_lineage - R2_global
df["delta_R2"] = df["R2_lineage"] - df["R2_global"]
top_genes = df.groupby("gene")["delta_R2"].mean().nlargest(20).index
print("Top lineage-specific genes:", list(top_genes))
print("Key CREs for those genes:\n",
      df[df["gene"].isin(top_genes)][["gene","peak_id","delta_R2"]].head(20))

# d) Regression vs correlation
corr_vs_beta = df[["beta_global","pearson_r"]].corr().iloc[0,1]
print(f"Correlation between β_global and |r|: {corr_vs_beta:.2f}")

# e) Activating vs repressing counts
print("Effect counts:\n", df["effect"].value_counts())

# f) Genes mainly regulated by repressors
dominant = df.groupby("gene")["effect"] \
             .agg(lambda s: s.value_counts().idxmax())
frac_repress = (dominant == "repressing").mean()
print(f"Fraction of genes primarily repressed: {frac_repress:.2f}")

# g) Location of repressors vs activators
print("Promoter enrichment:\n",
      pd.crosstab(df["effect"], df["is_promoter"], normalize="index"))
print("Intragenic enrichment:\n",
      pd.crosstab(df["effect"], df["is_intragenic"], normalize="index"))

# h) CREs with dual roles
dual = df.groupby("peak_id")["effect"].nunique().gt(1).sum()
print(f"CREs with both activating & repressing roles: {dual}")

# i) Re-cluster with directionality
pe_list = df["peak_id"].unique()
profiles = []
for pid in pe_list:
    sign = np.sign(df.loc[df["peak_id"]==pid,"beta_global"].mean())
    profiles.append(atac.loc[pid].values * sign)
Z = StandardScaler().fit_transform(np.array(profiles))
labels = KMeans(n_clusters=20, random_state=0).fit_predict(Z)
pd.DataFrame({"peak_id":pe_list,"signed_module":labels}) \
  .to_csv("data-processed/cre_signed_modules.csv", index=False)
print("Re-clustering with sign done.")